In [154]:

import json
import gensim
import pandas as pd
import nltk
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from gensim.models.ldamodel import LdaModel
from gensim.models import TfidfModel
from gensim.models import CoherenceModel
from nltk.tokenize import *
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from gensim import corpora
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ANAS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ANAS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [155]:
tweets_data_path = 'documentsfr.txt'

tweets_data = []
with open(tweets_data_path, "r",encoding="utf8") as tweets_file:
    
    for line in tweets_file:
        try:
            tweets_data.append(line)
        except:
            continue


In [156]:
tweets = pd.DataFrame()
tweets['text']=list(tweets_data)
doc_complete= tweets['text']
print(len(doc_complete))
print(doc_complete.head())

17560
0    ﻿En orbite,  VI-B est le second satellite lanc...
1    Le patron de Renault Carlos Ghosn voit sa gard...
2    Une rencontre avec les écoles canadiennes à  e...
3    Le  de gastronomie péruvienne s'invite à  http...
4    Pour éviter le froid, ces chiens abandonnés tr...
Name: text, dtype: object


In [157]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)



In [158]:
def clean(doc):
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    tokenizer =  RegexpTokenizer(r'\w+')
    stopw_fr = stopwords.words('french')
    
    token= [[token.lower() for token in tokenizer.tokenize(remove_emoji(document))] for document in doc ]
    stop_free = [[ t for t in doc if (t not in stopw_fr) & (t[:4] != 'http') & (not t.isdigit())]  for doc in token]
    normalize  = [[lemma.lemmatize(word) for word in p if (len(word)>3) & (len(word)<15) ] for p in stop_free]
    return normalize

In [159]:
doc_clean = clean(doc_complete)
#print(doc_clean)

dictionary = corpora.Dictionary(doc_clean)
dictionary.filter_extremes(no_below=2)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]


In [160]:
goodLdaModel = LdaModel(corpus=doc_term_matrix,id2word=dictionary,iterations=1000,num_topics=8,passes=350,
                        update_every=1,chunksize=7000,alpha='auto',per_word_topics=True)
print(goodLdaModel.print_topics(num_topics=8,num_words=5))

[(0, '0.060*"mondial" + 0.051*"ligne" + 0.006*"coopération" + 0.005*"renard" + 0.004*"arabie"'), (1, '0.014*"contre" + 0.006*"prison" + 0.005*"corruption" + 0.005*"football" + 0.005*"encore"'), (2, '0.016*"gouvernement" + 0.013*"président" + 0.009*"othmani" + 0.008*"cgem" + 0.007*"heure"'), (3, '0.009*"afrique" + 0.008*"bourita" + 0.007*"plan" + 0.006*"jerada" + 0.006*"ramadan"'), (4, '0.044*"candidature" + 0.008*"monde" + 0.008*"bank" + 0.008*"oncf" + 0.007*"demande"'), (5, '0.020*"sahara" + 0.011*"algérie" + 0.011*"procès" + 0.010*"hirak" + 0.009*"polisario"'), (6, '0.009*"train" + 0.009*"blessés" + 0.008*"boycott" + 0.008*"accident" + 0.008*"bouknadel"'), (7, '0.013*"migrant" + 0.009*"bourse" + 0.009*"gouvernement" + 0.008*"royale" + 0.007*"espagne"')]


In [161]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=goodLdaModel, texts=doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.6273656581464385


In [162]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(goodLdaModel, doc_term_matrix, dictionary)
vis

C:\Users\ANAS\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.019658 -0.260037       1        1  14.248693
2     -0.092307  0.126630       2        1  13.630538
4      0.103955  0.029915       3        1  13.228189
3      0.035670 -0.016228       4        1  13.054031
6      0.178229  0.039114       5        1  12.031864
7      0.003125  0.176817       6        1  11.930468
0     -0.268703 -0.030117       7        1  11.617410
1      0.020373 -0.066094       8        1  10.258804, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
581   Default  397.000000        mondial  397.000000  30.0000  30.0000
1152  Default  334.000000          ligne  334.000000  29.0000  29.0000
110   Default  333.000000    candidature  333.000000  28.0000  28.0000
657   Default  159.000000         sahara  159.000000  27.0000  27.0000
143   Default  201.000000   gouvernement  201.000000  26.0000  26.0000
124   Default  118.000000      président  118.000000  25.0000  25.0000
132   Default   96.000000        migrant   96.000000  24.0000  24.0000
825   Default  111.000000        algérie  111.000000  23.0000  23.0000
1467  Default   84.000000          hirak   84.000000  22.0000  22.0000
570   Default   95.000000         procès   95.000000  21.0000  21.0000
946   Default   75.000000      polisario   75.000000  20.0000  20.0000
72    Default   65.000000          train   65.000000  19.0000  19.0000
498   Default   64.000000        blessés   64.000000  18.0000  18.0000
198   Default  234.000000         contre  234.000000  17.0000  17.0000
265   Default   71.000000         bourse   71.000000  16.0000  16.0000
3819  Default   57.000000        boycott   57.000000  15.0000  15.0000
308   Default   84.000000           oncf   84.000000  14.0000  14.0000
1292  Default   73.000000           cgem   73.000000  13.0000  13.0000
491   Default  145.000000        afrique  145.000000  12.0000  12.0000
1127  Default   57.000000        bourita   57.000000  11.0000  11.0000
1256  Default   75.000000       accident   75.000000  10.0000  10.0000
1227  Default   53.000000      bouknadel   53.000000   9.0000   9.0000
1125  Default   55.000000           plan   55.000000   8.0000   8.0000
83    Default   59.000000         royale   59.000000   7.0000   7.0000
1325  Default   66.000000           bank   66.000000   6.0000   6.0000
145   Default  104.000000        othmani  104.000000   5.0000   5.0000
1259  Default   53.000000          heure   53.000000   4.0000   4.0000
600   Default   53.000000     commission   53.000000   3.0000   3.0000
86    Default   91.000000          trump   91.000000   2.0000   2.0000
404   Default   61.000000        demande   61.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
434    Topic8   11.212543          trans   11.979856   2.2108  -6.2504
661    Topic8   10.722733           huit   11.490026   2.2079  -6.2950
601    Topic8   10.557378         hasnaa   11.324566   2.2069  -6.3106
2565   Topic8   10.557166          crime   11.324578   2.2069  -6.3106
1889   Topic8   10.428878           sein   11.196306   2.2060  -6.3228
1966   Topic8   10.226363         devise   10.993664   2.2047  -6.3424
3478   Topic8    9.955807      exclusive   10.723206   2.2028  -6.3692
2948   Topic8    9.864939  cybersécurité   10.632100   2.2021  -6.3784
607    Topic8    9.827683    israélienne   10.594925   2.2019  -6.3822
405    Topic8   21.784967          ferme   25.233530   2.1301  -5.5862
2719   Topic8   19.851440        sénégal   22.946709   2.1321  -5.6791
603    Topic8   19.711962          lalla   22.803356   2.1314  -5.6862
255    Topic8   16.115208           week   18.753569   2.1254  -5.8876
1894   Topic8   20.116005     terrorisme   24.481602   2.0806  -5.6659
4906   Topic8   23.604994      chronique   32.460247   1.9585  -5.5059
700    To

In [163]:
print(goodLdaModel.print_topics(num_topics=7,num_words=10))

[(1, '0.014*"contre" + 0.006*"prison" + 0.005*"corruption" + 0.005*"football" + 0.005*"encore" + 0.004*"syrie" + 0.004*"membres" + 0.004*"chronique" + 0.004*"omar" + 0.004*"ferme"'), (7, '0.013*"migrant" + 0.009*"bourse" + 0.009*"gouvernement" + 0.008*"royale" + 0.007*"espagne" + 0.007*"service" + 0.007*"militaire" + 0.006*"assurance" + 0.006*"projet" + 0.006*"marine"'), (6, '0.009*"train" + 0.009*"blessés" + 0.008*"boycott" + 0.008*"accident" + 0.008*"bouknadel" + 0.007*"morts" + 0.007*"lion" + 0.006*"semaine" + 0.006*"force" + 0.006*"alerte"'), (0, '0.060*"mondial" + 0.051*"ligne" + 0.006*"coopération" + 0.005*"renard" + 0.004*"arabie" + 0.004*"santé" + 0.004*"marche" + 0.004*"afrique" + 0.004*"saoudite" + 0.004*"hervé"'), (2, '0.016*"gouvernement" + 0.013*"président" + 0.009*"othmani" + 0.008*"cgem" + 0.007*"heure" + 0.007*"commission" + 0.005*"contre" + 0.005*"mezouar" + 0.004*"faut" + 0.004*"groupe"'), (4, '0.044*"candidature" + 0.008*"monde" + 0.008*"bank" + 0.008*"oncf" + 0.007*